In [1]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix
data = pd.read_csv('train_set_100_2.csv', encoding='utf-8')
data['sucs'] = data['sucs'].values.astype(str)
data['ictihats'] = data['ictihats'].values.astype(str)
for i,suc in enumerate(data['sucs']):
    data['sucs'][i] = suc.replace(" ", "_")
print(data.sucs.unique())
data['labels'] = pd.factorize(data.sucs)[0]

['5607_sayılı_kanuna_muhalefet' 'undefined'
 'uyuşturucu_madde_ticareti_yapma' 'other' 'görevi_kötüye_kullanma'
 '6136_sayılı_yasaya_aykırılık' 'silahlı_terör_örgütüne_üye_olma'
 'kullanmak_için_uyuşturucu_madde_bulundurma'
 'kişiyi_hürriyetinden_yoksun_kılma' 'resmi_belgede_sahtecilik']


In [2]:
#data.head()
#data.iloc[:, 1] = data.iloc[:, 1].apply(lambda x: '__label__' + x)

In [3]:
#data split for train and test
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2, random_state=42)
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

In [4]:
train.head()

,ictihats,sucs,labels
0,hük mahkumiyet yerel mahkemeç bozma verile hük...,other,3
1,taraf muhakemes bölge adli mahkemes hukuk dair...,undefined,1
2,dav tapu şerhi terk istem ilişk 2797 kanun 672...,undefined,1
3,taraf görüle dava sonuç mahkemeç verile kabul ...,undefined,1
4,hük mahkumiyet yerel mahkemeç verile hüküm tem...,other,3


In [5]:
test.head()

,ictihats,sucs,labels
0,4857 41 46 57 taraf görüle dav sonuç verile te...,undefined,1
1,derecemahkemes mahkemes dav 3201 yas geçerli o...,undefined,1
2,hük mahkumiyet mahalli mahkemeç verile hük tem...,other,3
3,hük beraat yerel mahkemeç verile hük temyiz ed...,other,3
4,hük mahkumiyet dosya incelenerek düşünül büyük...,other,3


In [6]:
#Data prepare for fasttext format for train
train["label_format"]=0
train["label_format_without_ictihats"]=0
for i in range(len(train)):
    train.label_format[i]="__label__"+str(train.sucs[i])+" "+str(train.ictihats[i])
    train.label_format_without_ictihats[i]="__label__"+str(train.sucs[i])

C:\Users\onkol\AppData\Local\Temp/ipykernel_12484/1772977377.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["label_format"]=0
C:\Users\onkol\AppData\Local\Temp/ipykernel_12484/1772977377.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["label_format_without_ictihats"]=0
C:\Users\onkol\AppData\Local\Temp/ipykernel_12484/1772977377.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.or

In [7]:
#Data prepare for fasttext format for test
test["label_format"]=0
test["label_format_without_ictihats"]=0
for i in range(len(test)):
    test.label_format[i]="__label__"+str(test.sucs[i])+" "+str(test.ictihats[i])
    test.label_format_without_ictihats[i]="__label__"+str(test.sucs[i])

C:\Users\onkol\AppData\Local\Temp/ipykernel_12484/3952881165.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["label_format"]=0
C:\Users\onkol\AppData\Local\Temp/ipykernel_12484/3952881165.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["label_format_without_ictihats"]=0
C:\Users\onkol\AppData\Local\Temp/ipykernel_12484/3952881165.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/

In [8]:
train.label_format.to_csv('fasttext_train.txt',index=None,header=None)
test.label_format.to_csv('fasttext_test.txt',index=None,header=None)


In [9]:
import fasttext
#train classifier model
model = fasttext.train_supervised('fasttext_train.txt',epoch=100,lr=0.05,label_prefix='__label__',dim=300)
# model.save_model('model.bin')


In [10]:
model.test('fasttext_test.txt')
# print(model_test)
# precision = model_test[1]
# recall = model_test[2]
# f1_score = (2*(precision*recall))/(precision + recall)
# result = model.test('fasttext_train.txt')
# validation = model.test('fasttext_test.txt')
# text_line =  "accuracy:" + str(result[1])  + "\nvalidation:" + str(validation[1]) + '\n' 
# print ('Precision:', precision)
# print ('F1 score:', f1_score)
# print ('Recall:', recall)
# print(text_line)



(5571, 0.9545862502243763, 0.9545862502243763)

In [11]:

y_pred = test.iloc[:, 0].apply(lambda x: model.predict(x)[0][0])
#print(y_pred.unique())
y_test = test.label_format_without_ictihats
#print(y_test.unique())

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

                                                     precision    recall  f1-score   support

              __label__5607_sayılı_kanuna_muhalefet       0.70      0.67      0.68        94
              __label__6136_sayılı_yasaya_aykırılık       0.47      0.55      0.51        31
                    __label__görevi_kötüye_kullanma       0.53      0.45      0.49        20
         __label__kişiyi_hürriyetinden_yoksun_kılma       0.38      0.38      0.38        29
__label__kullanmak_için_uyuşturucu_madde_bulundurma       0.95      0.89      0.92        62
                                     __label__other       0.95      0.96      0.95      2576
                  __label__resmi_belgede_sahtecilik       0.76      0.61      0.68        36
           __label__silahlı_terör_örgütüne_üye_olma       0.98      0.96      0.97        49
                                 __label__undefined       0.99      0.99      0.99      2519
           __label__uyuşturucu_madde_ticareti_yapma       0.86      0

In [12]:
model.predict('bicaklamak')


(('__label__undefined',), array([1.00001001]))